In [8]:
import pandas as pd
import json

In [15]:
dfs = []
with open('../output.txt') as f:
    for line in f.readlines():
        json_data = pd.json_normalize(json.loads(line))
        dfs.append(json_data)
df = pd.concat(dfs, sort=False) # or sort=True depending on your needs

In [16]:
df

,client_name,execution_time,current_timestamp,state.cpu,state.ram,state.network_bandwidth,state.reliabilty,state.ExecutionTimes,state.RoundParticipation,state.clientName,train_output.accuracy,train_output.avg_epoch_loss
0,concrete-schema,21.665670,1.686608e+09,4.74,8.41,80.22,0.12738,+,+,concrete-schema,"[0.2808888888888889, 0.4057777777777778, 0.458]","[0.0, 0.0, 0.0]"
0,warm-color,21.743768,1.686608e+09,6.57,6.53,72.15,0.01010,+,+,warm-color,"[0.28944444444444445, 0.4151111111111111, 0.46...","[0.0, 0.0, 0.0]"
0,greedy-spitz,21.687510,1.686608e+09,5.41,9.34,75.98,0.05018,+,+,greedy-spitz,"[0.2763333333333333, 0.41555555555555557, 0.46...","[0.0, 0.0, 0.0]"
0,distinct-island,21.726048,1.686608e+09,3.73,6.29,20.96,0.04429,+,+,distinct-island,"[0.289, 0.4151111111111111, 0.4647777777777778]","[0.0, 0.0, 0.0]"
0,express-innovation,21.790720,1.686608e+09,5.21,10.19,6.75,0.14367,+,+,express-innovation,"[0.28655555555555556, 0.4017777777777778, 0.45...","[0.0, 0.0, 0.0]"
0,caramel-pinscher,21.804224,1.686608e+09,6.70,6.33,49.15,0.09559,+,+,caramel-pinscher,"[0.2876666666666667, 0.4023333333333333, 0.463...","[0.0, 0.0, 0.0]"
0,diagonal-sickbay,21.828089,1.686608e+09,4.28,7.08,69.73,0.06623,+,+,diagonal-sickbay,"[0.2827777777777778, 0.4048888888888889, 0.459...","[0.0, 0.0, 0.0]"
0,warm-weapon,21.823059,1.686608e+09,2.39,8.39,28.15,0.13008,+,+,warm-weapon,"[0.2867777777777778, 0.41488888888888886, 0.469]","[0.0, 0.0, 0.0]"
0,brownian-vault,21.826113,1.686608e+09,4.38,6.93,46.79,0.04615,+,+,brownian-vault,"[0.2801111111111111, 0.4087777777777778, 0.452...","[0.0, 0.0, 0.0]"
0,ebony-clique,21.878171,1.686608e+09,5.36,6.63,108.22,0.22492,+,+,ebony-clique,"[0.2827777777777778, 0.40844444444444444, 0.46...","[0.0, 0.0, 0.0]"
